In [1]:
import torch
import torch.nn as nn
import logging

from comblearn.data import DSFValueFunction
from comblearn.env import CombinatorialAuction

logging.basicConfig(level=20)

In [9]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
from torch.optim import Adam

In [10]:
device = 'cuda'

class MyModular(nn.Module):
    def __init__(self, k):
        super(MyModular, self).__init__()
        self.w = torch.randint(0, 5, (k, 1)).float().to(device)
        logging.info(f"{self.w.squeeze()}")

    def forward(self, x):
        return torch.matmul(x, self.w)


def social_welfare(ws, allocation):
    return torch.sum(torch.tensor([w(alloc) for w, alloc in zip(ws, allocation)]).to(device))

In [11]:
N = 5
m = 8
bidders = list(range(N))
q_init = 500
q_max = 505
items = list(range(m))

value_functions = [MyModular(m), 
                   MyModular(m), 
                   MyModular(m), 
                   MyModular(m), 
                   MyModular(m)]

value_functions_l = [DSFValueFunction(items, 110, [3], 300), 
                     DSFValueFunction(items, 110, [3], 300), 
                     DSFValueFunction(items, 110, [3], 300), 
                     DSFValueFunction(items, 110, [3], 300), 
                     DSFValueFunction(items, 110, [3], 300)]

INFO:root:tensor([3., 4., 1., 3., 4., 0., 0., 4.], device='cuda:0')
INFO:root:tensor([1., 1., 0., 0., 4., 0., 3., 0.], device='cuda:0')
INFO:root:tensor([2., 3., 4., 0., 0., 2., 3., 3.], device='cuda:0')
INFO:root:tensor([1., 2., 1., 3., 4., 0., 3., 2.], device='cuda:0')
INFO:root:tensor([2., 4., 2., 0., 3., 0., 0., 0.], device='cuda:0')


In [12]:
from torch.optim.optimizer import Optimizer, required

class PSGD(Optimizer):
    def __init__(self, params, omega=required, alpha=required):
        defaults = dict(omega=omega, alpha=alpha, iteration=0)
        super(PSGD, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(PSGD, self).__setstate__(state)

    def step(self, closure=None):
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            iteration = group['iteration']
            omega_k = group['omega'](iteration+1)
            alpha_k = group['alpha'](iteration+1)

            for p in group['params']:
                if p.grad is None:
                    continue
                d_p = p.grad.data
                param_state = self.state[p]
                if 'h' not in param_state:
                    param_state['h'] = torch.zeros_like(p)
                param_state['h'] = (1-omega_k) * param_state['h'] + omega_k * d_p
                h_k = param_state['h']
                p.data.add_(-alpha_k * h_k)
            
            group['iteration'] += 1
        return loss

In [13]:
from math import log2

opt_name = "PSGD"
omega = lambda k: 1/(log2(1 + k))
alpha = lambda k: 0.05/(k + log2(1 + k))
custom_optim = lambda p, lr: PSGD(p, omega=omega, alpha=alpha)

In [7]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter(log_dir=f"runs/{opt_name}")

In [ ]:
auction = CombinatorialAuction(bidders, items, value_functions, value_functions_l, q_init, q_max, custom_optim=custom_optim)
allocations, payments = auction.run(epochs=1000, lr=0.001, delta=0.005, sample_rate=5, writer=writer)